# **Importando pacotes**

In [1]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns


from sklearn.metrics 	     import mean_absolute_error, mean_squared_error
from sklearn.compose 	     import ColumnTransformer
from sklearn.pipeline 	     import Pipeline
from sklearn.model_selection import KFold, train_test_split, RandomizedSearchCV
from sklearn.preprocessing   import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute          import KNNImputer

from sklearn.linear_model    import LogisticRegression
from sklearn.svm             import SVC
from sklearn.tree            import DecisionTreeClassifier
from sklearn.ensemble        import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from xgboost                 import XGBClassifier
from lightgbm                import LGBMClassifier
from catboost                import CatBoostClassifier
from sklearn.dummy           import DummyClassifier
from sklearn.neighbors       import KNeighborsClassifier

from sklearn.feature_selection import SelectKBest
from category_encoders.target_encoder import TargetEncoder

import joblib

# **Classe**

In [2]:
class ModelCrafter:

    def __init__(self) -> None:
        
        self.models = dict()

        self.results = dict()

        self.results_per_fold = dict()

        self.kf = KFold(n_splits=5) 

    def AddModel(self, modelos : list = []) -> None:
        """Método para adicionar modelos ao objeto. A estrutura é uma lista de tuplas onde a tupla segue o seguinte esquema: (nome do modelo, modelo instanciado)"""
        
        for modelo in modelos:
            self.models[modelo[0]] = modelo[1]

    def RemoveModel(self, nome: str = None, tipo: str = None) -> None:
        """Remove modelos do objeto"""
        
        if tipo == 'all':
            self.models=dict()
            return

        del self.models[nome]

          
    def HoldOut(self,X: pd.DataFrame, y: pd.Series, pipe: Pipeline ,nome: str):
        """Treina somente um modelo dentre os que estão no objeto"""
        if nome not in self.models.keys():
            print('Modelo invalido')
            return
        
        if len(pipe.steps) > 1:
            pipe.steps.pop()
            
        pipe.steps.append((nome,self.models[nome]))

        modelo = pipe

        x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

        modelo.fit(x_train,y_train)
        predito = modelo.predict(x_test)

        aux_df = x_test.copy()
    

        return predito, y_test, aux_df

    def Validacao(self, X_train: pd.DataFrame = None, X_test: pd.DataFrame = None , y_train: pd.Series = None, y_test: pd.Series = None, pipe: Pipeline = None):
       
        if len(self.models) == 0:
            return "Nenhum modelo adicionado na estrutura"

        resultados = {'modelo':[],'mae_treino':[], 'mae_teste':[], 'rms_treino':[], 'rms_teste':[]}

        for aux in self.models.items():
            nome_modelo = aux[0]
            modelo = aux[1]

            if len(pipe.steps) > 1:
                pipe.steps.pop()

            
            print(f"-----{nome_modelo}-----")
            pipe.steps.append(("Model",modelo))
            modelo = pipe

            modelo.fit(X_train,y_train)

            pred_train = modelo.predict(X_train)
            pred_test = modelo.predict(X_test)


            mae_train  =  mean_absolute_error(y_train,pred_train)
            rms_train =  mean_squared_error(y_train,pred_train, squared = False)

            mae_test = mean_absolute_error(y_test,pred_test)
            rms_test = mean_squared_error(y_test, pred_test, squared= False)

            resultados['modelo'].append(nome_modelo)
            resultados['mae_treino'].append(mae_train)
            resultados['mae_teste'].append(mae_test)
            resultados['rms_treino'].append(rms_train)
            resultados['rms_teste'].append(rms_test)

        return pd.DataFrame(resultados)
        

    def ValidacaoCruzada(self, X: np.ndarray, y: np.array, pipe: Pipeline = None) -> None:
        """Treina todos os modelos inseridos no objeto através de validação cruzada"""
		
        if len(self.models) == 0:
            return "Nenhum modelo adicionado na estrutura"
        
        for aux in self.models.items():
            nome_modelo = aux[0]
            modelo = aux[1]

            if len(pipe.steps) > 1:
                pipe.steps.pop()

            mae = 0
            rms = 0
            
            
            resultados_aux = []

            print(f"-----{nome_modelo}-----")
            pipe.steps.append(("Model",modelo))
            modelo = pipe
            for i, (train_index, test_index) in enumerate(self.kf.split(X)):
                #print(f"Fold {i}:")
                
                #print(f"  Train: index={train_index}")
                #print(f"  Test:  index={test_index}")

                X_train = X.loc[train_index,:]
                y_train = y.loc[train_index]

                
                X_test = X.loc[test_index,:]
                y_test = y.loc[test_index]

                
                modelo.fit(X_train,y_train)

                predito = modelo.predict(X_test)
  
                
                resultados_aux.append((y_test,predito)) 
                
                mae  += mean_absolute_error(y_test,predito)
                rms += mean_squared_error(y_test,predito,squared = False)
               

                
            self.results[nome_modelo]=[mae/5, rms/5]
            self.results_per_fold[nome_modelo] = resultados_aux

        return self._gerar_resultado()

    def _gerar_resultado(self) -> None:
        """Gera os resultados em uma estrutura DataFrame"""
        
        indices = ['mae','rms']
        #display(pd.DataFrame(self.results,index=indices).T)
        return pd.DataFrame(self.results,index=indices).T

# **Carregando dados**

In [5]:
materials_raw = pd.read_csv('../datasets/project2/dataset_full.csv')

production = pd.read_csv('../datasets/project2/dataset_producao.csv')

In [6]:
materials_raw.head()

,Material,Space group,Crystal Type,Band gap,stoichiometry,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Be4,Pbcm,A-57-d,0.000000,A,4.0,4.000000,4,4,0.0,...,0.0,0.0,0.0,0.000000,37.710,37.710,37.71,37.71,0.000,0.000000
1,AlTe4,Cm,AB4-8-a,0.000000,AB4,32.5,44.200000,52,13,19.5,...,5.0,2.0,1.5,1.749286,46.200,40.680,55.40,37.00,9.200,10.728951
2,As4O6,P2_1,A2B3-4-a,3.876239,A2B3,20.5,18.000000,33,8,12.5,...,3.0,2.0,0.5,0.509902,17.520,15.064,29.80,5.24,12.280,12.523192
3,As4S6,Pc,A2B3-7-a,2.271995,A2B3,24.5,22.800000,33,16,8.5,...,3.0,2.0,0.5,0.509902,24.585,23.542,29.80,19.37,5.215,5.318277
4,B2N,P-3m1,AB2-164-bd,0.000000,AB2,6.0,5.666667,7,5,1.0,...,5.0,3.0,1.0,1.054093,14.065,16.220,20.53,7.60,6.465,6.814708


In [8]:
production.head()

,Material,Prototype,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,media_Electronegativity,media_pon_Electronegativity,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Sc2F3,P-3m1,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
1,Sc2F3,P-6m2,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
2,Sc2F3,Pmmn,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
3,Sc2F3,P1,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
4,Sc2Cl3,P-3m1,19.0,18.6,21,17,2.0,2.039608,2.26,2.440,...,9.0,1.0,4.0,4.079216,60.785,51.542,107.0,14.57,46.215,47.130237


# **Tratamento**

Vamos realizar algumas tratativas prévias antes mesmo de analisar os dados e treinar os modelos.

In [16]:
# Renomeando a coluna prototype para space group
production = production.rename({'Prototype':'Space group'},axis=1)
production

,Material,Space group,media_Z,media_pon_Z,max_Z,min_Z,desvio_Z,desvio_pon_Z,media_Electronegativity,media_pon_Electronegativity,...,max_NumberUnfilledOrbitals,min_NumberUnfilledOrbitals,desvio_NumberUnfilledOrbitals,desvio_pon_NumberUnfilledOrbitals,media_Polarizability,media_pon_Polarizability,max_Polarizability,min_Polarizability,desvio_Polarizability,desvio_pon_Polarizability
0,Sc2F3,P-3m1,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
1,Sc2F3,P-6m2,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
2,Sc2F3,Pmmn,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
3,Sc2F3,P1,15.0,13.8,21,9,6.0,6.118823,2.67,2.932,...,9.0,1.0,4.0,4.079216,55.350,45.020,107.0,3.70,51.650,52.672872
4,Sc2Cl3,P-3m1,19.0,18.6,21,17,2.0,2.039608,2.26,2.440,...,9.0,1.0,4.0,4.079216,60.785,51.542,107.0,14.57,46.215,47.130237
5,Sc2Cl3,P-6m2,19.0,18.6,21,17,2.0,2.039608,2.26,2.440,...,9.0,1.0,4.0,4.079216,60.785,51.542,107.0,14.57,46.215,47.130237
6,Sc2Cl3,Pmmn,19.0,18.6,21,17,2.0,2.039608,2.26,2.440,...,9.0,1.0,4.0,4.079216,60.785,51.542,107.0,14.57,46.215,47.130237
7,Sc2Cl3,P1,19.0,18.6,21,17,2.0,2.039608,2.26,2.440,...,9.0,1.0,4.0,4.079216,60.785,51.542,107.0,14.57,46.215,47.130237
8,Ti2F3,P-3m1,15.5,14.2,22,9,6.5,6.628725,2.76,3.004,...,8.0,1.0,3.5,3.569314,47.850,39.020,92.0,3.70,44.150,45.024342
9,Ti2F3,P-6m2,15.5,14.2,22,9,6.5,6.628725,2.76,3.004,...,8.0,1.0,3.5,3.569314,47.850,39.020,92.0,3.70,44.150,45.024342


In [19]:
# Removendo as colunas stoichiometry e Crystal Type
materials_raw = materials_raw.drop(['stoichiometry','Crystal Type'],axis=1)

Agora as variáveis de ambos os datasets estão iguais. Exceto pelo target presente no materials_raw e ausente do production

In [26]:
print(f"# variáveis do dataset materials_raw: {materials_raw.shape[-1]}",
      f"\n# variáveis do dataset production: {production.shape[-1]}")

# variáveis do dataset materials_raw: 105 
# variáveis do dataset production: 104


Verificando se os space group do dataset production estão presentes também no material_raw.

**R:** Todos estão presentes em ambos os datasets. Não será necessário realizar nenhum tratamento com relação a isso.

In [44]:
for space in production['Space group'].tolist():
    
    if space in materials_raw['Space group'].unique():
        print(f"{space} está presente em ambos")
    else:
        print(f"-->{space} não está presente em ambos<--")
    

P-3m1 está presente em ambos
P-6m2 está presente em ambos
Pmmn está presente em ambos
P1 está presente em ambos
P-3m1 está presente em ambos
P-6m2 está presente em ambos
Pmmn está presente em ambos
P1 está presente em ambos
P-3m1 está presente em ambos
P-6m2 está presente em ambos
Pmmn está presente em ambos
P1 está presente em ambos
P-3m1 está presente em ambos
P-6m2 está presente em ambos
Pmmn está presente em ambos
P1 está presente em ambos
